In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

Question 1

(a)

In [2]:
def f(ls):
    x = []
    for i in ls:
        x.append(str(i))
    return x

In [3]:
def Euler_disc(n, alpha, psi):
    S0 = 100.0
    r = 0.05
    T = 1.0
    v0_square = 0.04
    N = 50
    delta = T / N
    w1 = np.random.normal(0, 1, (n, N))
    w2 = np.random.normal(0, 1, (n, N))
    v_sq = np.zeros((n, N))
    v_sq[:, 0] = v0_square + alpha * v0_square * delta + psi * v0_square * np.sqrt(delta) * w2[:, 0]
    S = np.zeros((n, N))
    S[:, 0] = S0 + r * S0 * delta + np.sqrt(v0_square * delta) * w1[:, 0] * S0
    for i in xrange(1, N):
        S[:, i] = S[:, i - 1] + r * S[:, i - 1] * delta + np.sqrt(v_sq[:, i - 1] * delta) * w1[:, i] * S[:, i - 1]
        v_sq[:, i] = v_sq[:, i - 1] + alpha * v_sq[:, i - 1] * delta + psi * v_sq[:, i - 1] * np.sqrt(delta) * w2[:, i]
    return S[:, -1]

def common_Mont(n, K, alpha, psi):
    S = Euler_disc(n, alpha, psi)
    r = 0.05
    T = 1.0
    disc_factor = np.exp(-r * T)
    C = (S - K) * disc_factor
    C[C < 0] = 0
    c = C.mean()
    v = C.std() / np.sqrt(n)
    return (c, v)


paras = [[0.1, 0.1], [0.1, 1.0]]

pricing = np.array([list(common_Mont(10000, 100.0, i[0], i[1])) for i in paras])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=f(paras))

result    

,Price,Std
"[0.1, 0.1]",10.679813,0.152866
"[0.1, 1.0]",10.397055,0.153572


(b)

In [4]:
def black_scholes (s, k, t, v, r):

    d1 = np.log(s / k) / (v * np.sqrt(t)) + (r + (v ** 2) / 2) * np.sqrt(t) / v
    d2 = d1 - v * np.sqrt(t)
    pv = np.exp(-r * t)
    optprice = s * norm.cdf(d1) - k * pv * norm.cdf(d2)
    return optprice


def Euler_disc_V(n, alpha, psi):
    T = 1.0
    v0_square = 0.04
    N = 50
    delta = T / N
    w2 = np.random.normal(0, 1, (n, N))
    v_sq = np.zeros((n, N))
    v_sq[:, 0] = v0_square + alpha * v0_square * delta + psi * v0_square * np.sqrt(delta) * w2[:, 0]

    for i in xrange(1, N):
        v_sq[:, i] = v_sq[:, i - 1] + alpha * v_sq[:, i - 1] * delta + psi * v_sq[:, i - 1] * np.sqrt(delta) * w2[:, i]
    
    return v_sq.sum(axis=1) / N

def Avg_vol(n, alpha, psi):
    S = 100.0
    K = 100.0
    sigma = np.sqrt(Euler_disc_V(n, alpha, psi))
    T = 1.0
    r = 0.05
    C = black_scholes(S, K, T, sigma, r)
    c = C.mean()
    v = C.std() / np.sqrt(n)
    return (c, v)

    
paras = [[0.1, 0.1], [0.1, 1.0]]

pricing = np.array([list(Avg_vol(10000, i[0], i[1])) for i in paras])

result = pd.DataFrame({"Price": pricing[:, 0], "Std": pricing[:, 1]},
                     index=f(paras))

result       

,Price,Std
"[0.1, 0.1]",10.640567,0.002278
"[0.1, 1.0]",10.312163,0.022742


Question 2

(a)

In [5]:
def lambdas(alpha, tao, sigma, r):
    numer = 4 * alpha * np.exp(-alpha * tao)
    deno = sigma ** 2 * (1 - np.exp(-alpha * tao))
    return numer / deno * r
    
def r_calculate(sigma, alpha, tao, lamb, df, n):
    numer = sigma ** 2 * (1 - np.exp(-alpha * tao))
    deno = 4 * alpha
    chi_sq = np.random.noncentral_chisquare(df, lamb, n)
    return numer / deno * chi_sq

def CIR(n):
    alpha = .2
    sigma = .1
    b = .05
    r0 = .04
    N = 50
    d = 4 * b * alpha / sigma ** 2
    T = 1.0
    delta = T / N
    rs = np.zeros((n, N))
    lamb = lambdas(alpha, delta, sigma, r0)
    rs[:, 0] = r_calculate(sigma, alpha, delta, lamb, d, n)
    for i in xrange(1, N):
        lamb = lambdas(alpha, delta, sigma, rs[:, i - 1])
        rs[:, i] = r_calculate(sigma, alpha, delta, lamb, d, n)
    disc_factor = np.exp(-rs.sum(axis=1) * delta)
    return disc_factor

CIR(1000).mean(), CIR(1000).std() / np.sqrt(1000)

(0.96038748936875895, 0.00032859664479599872)

(b)

In [6]:
def lambdas(alpha, tao, sigma, r):
    numer = 4 * alpha * np.exp(-alpha * tao)
    deno = sigma ** 2 * (1 - np.exp(-alpha * tao))
    return numer / deno * r
    
def r_calculate(sigma, alpha, tao, lamb, df, n):
    numer = sigma ** 2 * (1 - np.exp(-alpha * tao))
    deno = 4 * alpha
    chi_sq = np.random.noncentral_chisquare(df, lamb, n)
    return numer / deno * chi_sq

def CIR_cap(n):
    alpha = .2
    delt = 1 / 12.0
    L = 1.0
    R = 0.05
    sigma = .1
    b = .05
    r0 = .04
    N = 50
    d = 4 * b * alpha / sigma ** 2
    T = 1.0
    delta = T / N
    rs = np.zeros((n, N))
    lamb = lambdas(alpha, delta, sigma, r0)
    rs[:, 0] = r_calculate(sigma, alpha, delta, lamb, d, n)
    for i in xrange(1, N):
        lamb = lambdas(alpha, delta, sigma, rs[:, i - 1])
        rs[:, i] = r_calculate(sigma, alpha, delta, lamb, d, n)
    cap = L * (rs[:, -1] - R) * delt
    cap[cap < 0] = 0
    
    
    return cap.mean(), cap.std() / np.sqrt(1000)

CIR_cap(1000)

(0.00038693999040142758, 2.5523544853084534e-05)

Questions 3

(a)

In [7]:
def call_price(ST, K, disc_factor):
    ST[ST < K] = K
    result = (ST - K) * disc_factor
    return result

In [8]:
def Resimulation_delta():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S01 = 90.0 + h
    S02 = 90.0 - h
    ST1 = S01 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    c1 = call_price(ST1, K, disc_factor)
    ST2 = S02 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    c2 = call_price(ST2, K, disc_factor)
    results = (c1 - c2) / (2 * h)
    return results.mean(), results.std() / np.sqrt(n)


print Resimulation_delta()

(0.2458006925636351, 0.0047571686029070012)


In [9]:
def Resimulation_delta_control():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    S01 = 90.0 + h
    S02 = 90.0 - h
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    ST1 = S01 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    c1 = call_price(ST1, K, disc_factor)
    ST2 = S02 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    c2 = call_price(ST2, K, disc_factor)
    results = (c1 - c2) / (2 * h)
    X = ST
    Y = results
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - EST)
    return c.mean(), c.std() / np.sqrt(n)

Resimulation_delta_control()

(0.23879716038564139, 0.0030960732097133313)

In [10]:
def Resimulation_vega():
    K = 100
    r = .1
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    sigma1 = .25 + h
    sigma2 = .25 - h
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    ST1 = S0 * np.exp((r - .5 * sigma1 ** 2) * T + sigma1 * np.sqrt(T) * Zs)
    c1 = call_price(ST1, K, disc_factor)
    ST2 = S0 * np.exp((r - .5 * sigma2 ** 2) * T + sigma2 * np.sqrt(T) * Zs)
    c2 = call_price(ST2, K, disc_factor)
    results = (c1 - c2) / (2 * h)
    
    return results.mean(), results.std() / np.sqrt(n)

Resimulation_vega()

(12.165904403113496, 0.27553051183209304)

In [11]:
def Resimulation_vega_control():
    K = 100
    r = .1
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    sigma1 = .25 + h
    sigma2 = .25 - h
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    ST1 = S0 * np.exp((r - .5 * sigma1 ** 2) * T + sigma1 * np.sqrt(T) * Zs)
    c1 = call_price(ST1, K, disc_factor)
    ST2 = S0 * np.exp((r - .5 * sigma2 ** 2) * T + sigma2 * np.sqrt(T) * Zs)
    c2 = call_price(ST2, K, disc_factor)
    results = (c1 - c2) / (2 * h)
    X = ST
    Y = results
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - EST)
    
    return c.mean(), c.std() / np.sqrt(n)

Resimulation_vega_control()

(12.703390987818013, 0.17566544566502951)

In [12]:
def Greek_delta_Pathwise():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    results = (ST > K) * ST / S0 * disc_factor
    return results.mean(), results.std() / np.sqrt(n)


print Greek_delta_Pathwise()

(0.23656347375004239, 0.0046796627753522609)


In [13]:
def Greek_delta_Pathwise_control():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    results = (ST > K) * ST / S0 * disc_factor
    X = ST
    Y = results
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - EST)
    
    return c.mean(), c.std() / np.sqrt(n)


print Greek_delta_Pathwise_control()

(0.24038374592947157, 0.0031252711389849145)


In [14]:
def Greek_vega_Pathwise():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    ST_selected = np.zeros(len(ST))
    ST_selected[ST > K] = ST[ST > K]
    ST_selected[ST < K] = S0
    ST_re = (np.log(ST_selected / S0) - (r + 0.5 * sigma ** 2 - delt) * T) / sigma
    results = disc_factor * ST_re * ST_selected
    results[ST < K] = 0
#     EST = S0 / disc_factor
#     X = ST
#     Y = results
#     sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
#     sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
#     a_hat = -sig_xy / sig_x ** 2
#     c = Y + a_hat * (X - EST)
    return results.mean(), results.std() / np.sqrt(n)

Greek_vega_Pathwise()

(12.618306502880884, 0.28039731338942558)

In [15]:
def Greek_vega_Pathwise_Control():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    ST_selected = np.zeros(len(ST))
    ST_selected[ST > K] = ST[ST > K]
    ST_selected[ST < K] = S0
    ST_re = (np.log(ST_selected / S0) - (r + 0.5 * sigma ** 2 - delt) * T) / sigma
    results = disc_factor * ST_re * ST_selected
    results[ST < K] = 0
    EST = S0 / disc_factor
    X = ST
    Y = results
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - EST)
    return c.mean(), c.std() / np.sqrt(n)

Greek_vega_Pathwise_Control()

(13.118377327037928, 0.18006531746043808)

In [16]:
def log_likelihood_delta():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    H = ST - K
    H[H < 0] = 0
    temp = (np.log(ST / S0) - (r - .5 * sigma ** 2) * T) / (S0 * sigma ** 2 * T)
    c = disc_factor * H * temp
    return c.mean(), c.std() / np.sqrt(n) 
    
log_likelihood_delta()    

(0.24449846228376559, 0.0087183960741037177)

In [17]:
def log_likelihood_delta_control():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    H = ST - K
    H[H < 0] = 0
    temp = (np.log(ST / S0) - (r - .5 * sigma ** 2) * T) / (S0 * sigma ** 2 * T)
    result = disc_factor * H * temp
    X = ST
    Y = result
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - EST)
    return c.mean(), c.std() / np.sqrt(n) 
    
log_likelihood_delta_control()    

(0.24156258280939083, 0.0065948759096517351)

In [18]:
def log_likelihood_vega():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    H = ST - K
    H[H < 0] = 0
    d = (np.log(ST / S0) - (r - .5 * (sigma ** 2) - delt) * T) / (sigma * np.sqrt(T))
    d_prime = ((np.log(S0 / ST) + (r - delt + sigma ** 2) * T)) / (sigma ** 2 * np.sqrt(T))
    temp = -d * d_prime - 1 / sigma
    c = disc_factor * H * temp
    return c.mean(), c.std() / np.sqrt(n) 
    
log_likelihood_vega()    

(13.885495490621144, 0.75903076115262436)

In [19]:
def log_likelihood_vega_control():
    K = 100
    r = .1
    delt = .03
    sigma = .25
    T = .2
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 90.0
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    H = ST - K
    H[H < 0] = 0
    d = (np.log(ST / S0) - (r - .5 * (sigma ** 2) - delt) * T) / (sigma * np.sqrt(T))
    d_prime = ((np.log(S0 / ST) + (r - delt + sigma ** 2) * T)) / (sigma ** 2 * np.sqrt(T))
    temp = -d * d_prime - 1 / sigma
    result = disc_factor * H * temp
    X = ST
    Y = result
    sig_x = np.sum((X - X.mean()) ** 2 / (len(X) - 1)) ** 0.5
    sig_xy = np.cov(X, Y)[0, 1] * len(Y) / (len(Y) - 1)
    a_hat = -sig_xy / sig_x ** 2
    c = Y + a_hat * (X - EST) 
    return c.mean(), c.std() / np.sqrt(n) 
    
log_likelihood_vega_control()    

(12.67002229736859, 0.65604368182614747)

Question 4

In [20]:
from scipy.stats import norm

In [21]:
def digital_option_close_form():
    K = 100
    r = .05
    sigma = .2
    T = 1.0
    S0 = 95.0
    disc_factor = np.exp(-r * T)
    deno = sigma * S0 * np.sqrt(T)
    x = (np.log(K / S0) - (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d1 = np.log(S0 / K) / (sigma * np.sqrt(T)) + (r + (sigma ** 2) / 2) * np.sqrt(T) / sigma
    d2 = d1 - sigma * np.sqrt(T)
    price = disc_factor * (1 - norm.cdf(x))
    Nd2_prime = 1 / np.sqrt(2 * np.pi) * np.exp(-0.5 * d2 ** 2)
    delta = disc_factor * Nd2_prime / deno
    return price, delta

digital_option_close_form()

(0.43528841364206289, 0.019860050706032074)

In [22]:
def digital_pay_off(ST, K, disc_factor):
    result = (ST > K) * disc_factor
    return result
    

In [23]:
def Resimulation_delta_digital():
    K = 100
    r = .05
    delt = .03
    sigma = .2
    T = 1.0
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S01 = 95.0 + h
    S02 = 95.0 - h
    ST1 = S01 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    c1 = digital_pay_off(ST1, K, disc_factor)
    ST2 = S02 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    c2 = digital_pay_off(ST2, K, disc_factor)
    results = (c1 - c2) / (2 * h)
    return results.mean(), results.std() / np.sqrt(n) 


print Resimulation_delta_digital()

(0.0, 0.0)


We can see that given the sample size is samll(only 10000, using this way we cannot get a good result.)

In [24]:
def log_likelihood_delta_digital():
    K = 100
    r = .05
    delt = .03
    sigma = .2
    T = 1.0
    n = 10000
    h = .0001
    disc_factor = np.exp(-r * T)
    Zs = np.random.normal(0, 1, n)
    S0 = 95.0
    EST = S0 / disc_factor
    ST = S0 * np.exp((r - .5 * sigma ** 2) * T + sigma * np.sqrt(T) * Zs)
    H = ST > K
    temp = (np.log(ST / S0) - (r - .5 * sigma ** 2) * T) / (S0 * sigma ** 2 * T)
    c = disc_factor * H * temp
    return c.mean(), c.std() / np.sqrt(n) 
    
log_likelihood_delta_digital()    

(0.019931077701920059, 0.00029512178135787849)